<a href="https://colab.research.google.com/github/Araslt/spark/blob/main/LD02_ipynb_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=e0d966b0169ae91e716565ca947deed3e34b4b5cfe718773001e2be805894f3d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
# SparkContext yra pagrindinė klasė, kuri valdo Spark aplikaciją ir nustato ką ji turėtų daryti.
# SparkConf yra klasė, kuri leidžia konfigūruoti Spark aplikacijos nustatymus.
# 6ios klasės importuojamos, kad būtų galima sukurti SparkContext objektą, kuris naudojamas valdyti Spark aplikaciją ir SparkConf objektą
from pyspark import SparkContext, SparkConf
# sc.stop()  # Stop the current SparkContext object
conf = SparkConf().setAppName('MyApp')
sc = SparkContext(conf=conf)

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

################## **LD02** #############################
3. Sudaryta lentele, kurioje matosi kiek pristatyta siuntu ("siuntu skaicius")
bei aptarnauta klientu ("Sustojimo klientu skaicius") geografinese
zonose ("geografine zona") skirtingomis savaites dienomis ("sustojimo
savaites diena").



In [4]:
# nuskaitomas  failo turinys su Spark tekstinių failų funkcija
# sc.textFile() ir jis išsaugomas kintamajame (RDD objekte) A.
A = sc.textFile("duom_cut.txt")

# funkcija gauna eilutę iš tekstinio failo kaip argumentą ir
# grąžina sąrašą (listą) iš eilučių, kurių kiekviena eilutė yra tekstas tarp }}{{.
# gaunamas maždaug toks outputas: ['a}{b}{c}{d', 'e'], jei buvo toks "{{a}{b}{c}{d}}{{e}}"
def myfmap(line):
    line = line.strip()
    line = line[2:len(line)-2]
    return line.split('}}{{')

In [5]:
# funkcija gauna atskiras eilutes, kurios yra gaunamos iš ankstesnės funkcijos,
# kaip argumentą ir iš jų išskiria klientų skaičių, savaitės dieną,
# geografinę zoną ir siuntų skaičių. Tuomet funkcija susikuria du tuple:
# pirmasis yra unikalus raktas, susidedantis iš zonos ir savaitės dienos,
# o antrasis yra tuple su siuntų skaičiumi ir klientų skaičiumi.
# Funkcija grąžina None, jei nebuvo rasti kliento, savaitės dienos,
# zonos ar siuntos duomenys.
# Funkcijos mymap() argumentas "stopas" yra funkcijos myfmap() gražinama reikšmė.
def mymap(stopas):
    klientai = 0
    diena = 0
    zona = None
    siunta = 0

    parstrings = stopas.split('}{')
    for parstring in parstrings:
        (vardas, reiksme) = parstring.split('=')
        if(reiksme != ''):
            if(vardas == 'Sustojimo klientu skaicius'):
                klientai=int(reiksme)
            if(vardas == 'sustojimo savaites diena'):
                diena=int(reiksme)
            if(vardas == 'siuntu skaicius'):
                siunta=int(reiksme)
            if(vardas == 'geografine zona'):
                zona=reiksme

    if(klientai != 0 and diena is not None and zona is not None and siunta != 0):
        return ((zona, diena), (siunta, klientai))
    else:
        return None


In [6]:
# flatMap(myfmap) funkcija naudoja anksčiau apibrėžtą myfmap funkciją suskaidyti
# duomenis pagal }}{{ simbolius.
# map(mymap) funkcija naudoja anksčiau apibrėžtą mymap funkciją,
# kad apdorotų kiekvieną iš gautų eilučių.
# filter(lambda x: x is not None) funkcija atmetė tuos duomenis,
# kuriuose nebuvo rasti klientų, savaitės dienos, zonos ar siuntų duomenys.
# reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])) funkcija sumažina duomenų kiekį,
# apjungdama tokius tuple, kurių raktai sutampa ir išsaugo tų tuple reikšmes,
# sumuojant jų siuntų ir klientų skaičių.
ATS = A.flatMap(myfmap)\
    .map(mymap)\
    .filter(lambda x: x is not None)\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))


# Išvesti rezultatus
print(ATS.take(5))

[(('Z1', 3), (618, 158)), (('Z1', 5), (232, 85)), (('Z3', 5), (9, 4)), (('Z1', 1), (267, 81)), (('Z3', 1), (8, 2))]


In [7]:
import pandas as pd
# sugeneruoti lentelės duomenis iš Spark rezultatų
table_data = []
for ((zona, diena), (siunta, klientai)) in ATS.collect():
    row = {'zona': zona, 'savaites diena': diena, 'siuntu sk.': siunta, 'Sustojimo klientu sk.': klientai}
    table_data.append(row)

# sukurti pandas lentelę su gautais duomenimis
df = pd.DataFrame(table_data)
print(df)




   zona  savaites diena  siuntu sk.  Sustojimo klientu sk.
0    Z1               3         618                    158
1    Z1               5         232                     85
2    Z3               5           9                      4
3    Z1               1         267                     81
4    Z3               1           8                      2
5    Z3               3          40                     16
6    Z1               2         607                    137
7    Z1               0          92                     18
8    Z1               4         368                     91
9    Z3               0           4                      3
10   Z3               2          17                      8
11   Z3               4          10                      6


In [8]:
# substitute
def mymap(stopas):
    mapping = {}
    parstrings = stopas.split('}{')
    for parstring in parstrings:
        (vardas, reiksme) = parstring.split('=')
        if reiksme:
            mapping[vardas] = reiksme
    if all(k in mapping for k in ['Sustojimo klientu skaicius', 'sustojimo savaites diena', 'siuntu skaicius', 'geografine zona']):
        return ((mapping['geografine zona'], int(mapping['sustojimo savaites diena'])), (int(mapping['siuntu skaicius']), int(mapping['Sustojimo klientu skaicius'])))
    else:
        return None
# print(mymap('geografine zona=1}{sustojimo savaites diena=2}{siuntu skaicius=3}{Sustojimo klientu skaicius=4'))



################## **END LD02** #############################